# Introduction to Spark

In this notebook, you will be introduced to the Apache Spark libary for big data processing. There's a now a python package called `pyspark` which will load Spark for you. Tthe variable `sc` is a Spark context that lets you interact with the Spark runtime. Check that it is correctly initialised:

In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print(sc)
print("Ready to go!")

<SparkContext master=local[*] appName=pyspark-shell>
Ready to go!


### Learning activity: Create RDDs

To analyse large datasets using Spark you will load them into Resilient Distributed Datasets (RDDs). There are a number of ways in which you can create RDDs. Use the `parallelize()` function to create one from a Python collection of elements ["Hello", "World", "!"] 

In [2]:
l = ["Hello", "World", "!"]
sc.parallelize(l)

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:194

Use the `textFile()` function to create an RDD from the file `war-and-peace.txt` and store it in a variable called `lines`. 

In [3]:
lines = sc.textFile("war-and-peace.txt")

### Learning activity: Basic RDD manipulation

Print the number of lines in War and Peace using the functions `count()`.

In [4]:
lines.count()

54223

Print the first 15 lines using the functions `take()`.

In [5]:
lines.take(15)

['                                      1869',
 '                                 WAR AND PEACE',
 '                                 by Leo Tolstoy',
 'BK1',
 '                                 BOOK ONE: 1805',
 'BK1|CH1',
 '  CHAPTER I',
 '',
 '  "Well, Prince, so Genoa and Lucca are now just family estates of the',
 "Buonapartes. But I warn you, if you don't tell me that this means war,",
 'if you still try to defend the infamies and horrors perpetrated by',
 'that Antichrist- I really believe he is Antichrist- I will have',
 'nothing more to do with you and you are no longer my friend, no longer',
 "my 'faithful slave,' as you call yourself! But how do you do? I see",
 'I have frightened you- sit down and tell me all the news."']

### Learning activity: `filter()` and `map()` and `distinct()`

Lets apply some transformations onto RDDs. The following helper function will be useful to select the words from a line.

In [6]:
# A helper function to compute the list of words in a line of text
import re
def get_words(line):
    return re.compile('\w+').findall(line)

print(get_words("This, is a test!"))

['This', 'is', 'a', 'test']


Use `filter()` to count the number of lines which mention `war` and the number of lines which mention `peace`.

In [7]:
# How often are war and peace mentioned?
war_lines = lines.filter(lambda l:"war" in get_words(l))
peace_lines = lines.filter(lambda l: "peace" in get_words(l))

print("war lines: {} and peace lines: {}" .format(war_lines.count(), peace_lines.count()))

war lines: 265 and peace lines: 104


Use `map()` to capitalise each line in the RDD, and print the first 15 capitalized lines.

In [8]:
# Capitalize each line in the RDD
cap_lines =lines.map(lambda l: l.upper())
cap_lines.take(15)

['                                      1869',
 '                                 WAR AND PEACE',
 '                                 BY LEO TOLSTOY',
 'BK1',
 '                                 BOOK ONE: 1805',
 'BK1|CH1',
 '  CHAPTER I',
 '',
 '  "WELL, PRINCE, SO GENOA AND LUCCA ARE NOW JUST FAMILY ESTATES OF THE',
 "BUONAPARTES. BUT I WARN YOU, IF YOU DON'T TELL ME THAT THIS MEANS WAR,",
 'IF YOU STILL TRY TO DEFEND THE INFAMIES AND HORRORS PERPETRATED BY',
 'THAT ANTICHRIST- I REALLY BELIEVE HE IS ANTICHRIST- I WILL HAVE',
 'NOTHING MORE TO DO WITH YOU AND YOU ARE NO LONGER MY FRIEND, NO LONGER',
 "MY 'FAITHFUL SLAVE,' AS YOU CALL YOURSELF! BUT HOW DO YOU DO? I SEE",
 'I HAVE FRIGHTENED YOU- SIT DOWN AND TELL ME ALL THE NEWS."']

Use `flatMap()` to create an RDD of the words in War and Peace and count the number of words.

In [9]:
# Split each line into words using get_words()
words = lines.flatMap(lambda l: get_words(l))
words.take(5)

['1869', 'WAR', 'AND', 'PEACE', 'by']

Finally, use `distinct()` to count the number of different words in the RDD.

In [10]:
# Count the number of distinct words
distinct_words = words.distinct()
distinct_words.count()

19206

### Learning activity: Set like transformations

Use the functions `union()` and `intersection()` to create RDDs of lines with either war or peace mentioned, and both war and peace being mentioned. Count how many lines of each type there are and print some examples.

In [11]:
union_lines = war_lines.union(peace_lines)
intersection_lines = war_lines.intersection(peace_lines)

print("Union Lines: {} and Intersection {}".format(union_lines.count(), intersection_lines.count()))

Union Lines: 369 and Intersection 7


### Learning activity: `reduce()`

You have already seen three actions: `collect()` which returns all elements in the RDD, `take(n)`, which return the first `n` elements of the RDD, and `count()` which returns the number of elements in the RDD.

The action `reduce()` takes as input a function which collapses two elements into one. Use it to find the longest word in War and Peace.

In [12]:
longest_word = words.reduce(lambda a,b: a if len(a) >= len(b) else b)
print(longest_word)


characteristically


### Bonus activity: merging filters

Find all the lines that mention both war and peace **without** using `intersection()`.

In [15]:
inter_lines = (lines.filter(lambda l: "war" in get_words(l) and "peace" in get_words(l)
                           )
              )
inter_lines.count()
#print(inter_lines)

7

### Bonus learning activity: Finding proper nouns

The Python function `str.istitle()` returns `True` if the string `str` is titlecased: the first character is uppercase and others are lowercase. Use it to:
* Find the set of distinct words in War and Peace which are titlecased
* Find the set of distinct words in War and Peace which are not titlecased

In [61]:
distinct_words = words.distinct()

print(distinct_words.count())

r = distinct_words.collect()
r = list(r)

titlecased = []
nottitlecased = []
for i in range(1,distinct_words.count()):
    if r[i].istitle() == True:
        titlecased.append(1)
    else:
        nottitlecased.append(0)
        
print("Number of distinct title cased words: ", len(titlecased))
print("Number of distinct not title cased words: ", len(nottitlecased))

19206
Number of distinct title cased words:  3068
Number of distinct not title cased words:  16137


The Python function `str.lower` returns a string with all characters of `str` lowercase. Use it, along with your previously generated RDD to find the set of words in War and Peace which only appear titlecased.

In [92]:
distinct_words = words.distinct()

print(distinct_words.count())

r = distinct_words.collect()
r = list(r)

titlecased = []
nottitlecased = []
for i in range(1,distinct_words.count()):
    if r[i].islower() == True:
        titlecased.append(1)
    else:
        nottitlecased.append(0)
        
print("Number of distinct lowercase words: ", len(titlecased))
print("Number of distinct not lowercase words: ", len(nottitlecased))

19206
Number of distinct lowercase words:  15940
Number of distinct not lowercase words:  3265


In [93]:
print("That means some of the words contains uppercases characters but are not titlecased")

That means some of the words contains uppercases characters but are not titlecased


# Key/Value pairs in Spark

### Learning activity: WordCount in Spark

Use the functions `flatMap()` and `reduceByKey()` to count the number of occurences of each word in War and Peace, and print the count of five words.

In [16]:
#words = data.flatMap(lambda: get_words(line))
word_pairs = words.map(lambda word: (word,1))
word_counts = word_pairs.reduceByKey(lambda c1, c2: c1+c2)
word_counts.take(10)

[('PEACE', 1),
 ('Leo', 1),
 ('BOOK', 15),
 ('ONE', 2),
 ('1805', 24),
 ('Prince', 1578),
 ('Genoa', 3),
 ('Lucca', 2),
 ('are', 1222),
 ('now', 1158)]

### Learning activity: using `groupByKey()`

Reimplement the above word count using `groupByKey()` instead of `reduceByKey()`

In [17]:
word_pairs = words.map(lambda word: (word,1))
#test = word_pairs.groupByKey()
#test.take(5)

test = word_pairs.groupByKey().map(lambda pair: (pair[0], sum(pair[1])) )
test.take(5)

[('PEACE', 1), ('Leo', 1), ('BOOK', 15), ('ONE', 2), ('1805', 24)]

### Bonus learning activity: computing the average of each key

The pair RDD defined below `word_line_pairs` has an element for each line in War and Peace with as key the first word, and as value the line itself. Use it to compute the average length of each line for each starting word.

In [18]:
word_line_pairs = lines.filter(lambda line: len(get_words(line)) > 0).map(lambda line: (get_words(line)[0], line))     

In [73]:
lenght = word_line_pairs